INVESTIGATING OR ANALYSING MATERIALS BY DETERMINING THEIR CHEMICAL OR PHYSICAL PROPERTIES

I – Term detection

1-1	What do we call as a term?

It a phrase or multiword or a word that have

-	A clear and specific meaning within the context of the patent

-	It should not a common word or phrase found in everyday language

-	A technical or scientific concept, material, method or device that is specific to the invention described in the patent.

1-2	Create a gold dataset

1-3	Build a rule-based baseline (may using re or generating n-grams and filter it)

1-4	Train a statistical model (bases on spacy)

1-5	Improve annotation using prodigy

1-6	Evaluate the model 

1-7	Iterate and refine the rule-based line


In [8]:
import sys
import time
import json
import pprint
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'3.5.1'

In [9]:
patent_data = open('G01N.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

In [10]:
len(patent_texts)

2139

In [11]:
def get_feature_names(vectorizer):
    if hasattr(vectorizer, 'get_feature_names'):
        return vectorizer.get_feature_names()
    else:
        return vectorizer.get_feature_names_out()

In [12]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes), 'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


In [13]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(
    1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X = cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=get_feature_names(cvectorizer)).T.sort_values(by=0, ascending=False)
termdf_cv.head(25)

c:\Users\junio\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
c:\Users\junio\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0
amino acid,24339
nucleic acid,13185
amino acid sequence,11407
light source,6943
amino acids,6170
control unit,5249
nucleotide sequence,4374
variable region,3264
mass spectrometry,3091
monoclonal antibody,2917


In [14]:
termdf_cv[termdf_cv[0] > 500]
termdf_cv.to_csv('terms.tsv', sep='\t')

In [16]:
 #!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/587.7 MB 4.0 MB/s eta 0:02:29
     ---------------------------------------- 0.2/587.7 MB 3.0 MB/s eta 0:03:15
     ---------------------------------------- 0.4/587.7 MB 2.8 MB/s eta 0:03:31
     ---------------------------------------- 0.6/587.7 MB 3.2 MB/s eta 0:03:07
     ---------------------------------------- 0.7/587.7 MB 3.1 MB/s eta 0:03:11
     ---------------------------------------- 0.8/587.7 MB 3.1 MB/s eta 0:03:13
     ---------------------------------------- 0.9/587.7 MB 2.7 MB/s eta 0:03:36
     ---------------------------------------- 1.0/587.7 MB 2.6 MB/s eta 0:03:44
     ---------------------------------------- 1.0/587.7 MB 2.5 MB/s eta 0:03:53
     ---------------------------------------- 1.1/587.7 MB 2.4 MB/s eta 0:04:01
     ---------------------------------------- 1.2/587.7 MB 2.4 MB/s eta 0:04:07
     ---------------------------------------- 1

In [18]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][10000:12000])
#displacy.render(doc, style="ent", jupyter=True)

In [20]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]
matcher.add("PHYS", patterns)

In [28]:
train_lines, test_lines = train_test_split(
    patent_lines, test_size=0.3, random_state=42)


print(len(train_lines))
print(len(test_lines))

231415
99179


In [ ]:
def create_dataset(text, n_lines, filename, offset=0):
    LABEL = "PHYS"
    doc_bin = DocBin()  # create a DocBin object

    for training_example in tqdm(text[offset:offset+n_lines]):
        doc = nlp.make_doc(training_example)
        ents = []

        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=LABEL)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents
        doc_bin.add(doc)
    doc_bin.to_disk(filename)


In [ ]:
create_dataset(train_lines, 60000, "training_data.spacy")
create_dataset(test_lines, 2000, "valid_data.spacy")

In [ ]:
# Run to generate full training config
# !python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
# !python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy --gpu-id 0


In [ ]:
# Load best model
nlp= spacy.load("./spacy_output/model-best")

# Just text snippet
doc = nlp("Nucleoside mimetic includes those structures used to replace the sugar or the sugar and the base and not necessarily the linkage at one or more positions of an oligomeric compound \
              such as for example nucleoside mimetics having morpholino,cyclohexenyl, cyclohexyl, tetrahydropyranyl, bicyclo or tricyclo sugar mimetics e.g. non furanose sugar \
              units.Nucleotide  means a nucleoside having a phosphate \
              group covalently linked to the sugar portion of the nucleoside.")

# Show NER results
spacy.displacy.render(doc, style="ent", jupyter=True)